# **PASOS:**



1.   PASO 1:

> Asegurate de tener un entorno de ejecución de GPU porque en CPU los modelos de AI corren peor y tienden a ser menos eficaces

2.   PASO 2:
> Sube los audios que deseas transcribir a la sección de 'Archivos'

3.   PASO 3:
> En la funcion 'unit_test' en la ultima parte del codigo, se puede seleccionar el idioma 'languages' como el modelo (mayor presición mas grande y pesado será el modelo [base, medium, large]).
**IMPORTANTE:** Asegurate que el primer parámetro 'audio_path' tenga el nombre exacto (tal cual está en la seccion de archivos). si no está con el mismo nombre se puede renombrar el archivo o cambiar el nombre en el propio código


```
def unit_test():
    txt = STTModels().faster_whisper(
        "audio.wav",
        # es: español , en: ingles, pt: portugues
        language="es",
        model="large", # large for better transcriptions
        compute_type="float32",
        beam_size=5,
    )
```

Algunas consideraciones: El tiempo de ejecución puede tardar mas si se elige un modelo mas grande, si se elige le modelo base, solo la descarga puede tardar hasta 1 minuto pero si se elige un modelo grande, solo en descargar el modelo puede tardar 5 minutos. (Pero una vez descargado el modelo ya tardará menos en transcribir los audios)

4.   PASO 4:
> Ejecuta todo (Ctrl + F9). Puedes dar click en el botón 'Entornos de ejecución' y luego click en 'Ejecutar todo'


Instalar las librerias necesarias

In [2]:
!pip install -q deprecated pydantic whisper-timestamped faster-whisper torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.2/803.2 kB 55.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 78.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 79.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 58.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.9 MB/s eta

Importar modulos

In [3]:
from deprecated import deprecated
from typing import ClassVar, Literal
from pydantic import HttpUrl
import whisper
from faster_whisper import WhisperModel
from whisper import _MODELS
from whisper.tokenizer import LANGUAGES
import torch
import os

Se verifica si se tiene acceso a una GPU. Importante para que el modelo de AI tenga recursos físicos donde ejecutarse

In [6]:
a = torch.cuda.is_available()
print(a)

True


Código

In [8]:
"""STT Models"""

class TTSModels:
    pass


class STTModels:
    """
    Speech-to-Text / Automatic Speech Recognition
    """

    def __init__(self):
        self.model_cache = {}

    # List of whisper models available
    whisper_models: ClassVar[list[str]] = list(_MODELS.keys())
    whisper_languages: ClassVar[list[str]] = list(LANGUAGES.keys())
    whisper_docs: ClassVar[HttpUrl] = "https://github.com/openai/whisper"
    compute_types: ClassVar[list[str]] = ["int8", "float16", "float32"]

    @deprecated(reason="Use the 'faster_whisper' for speed and efficiency")
    def whisper(self, audio_path: str, language: str, model: str = "tiny"):
        """
        Whisper model

        Args:
            model (str): The whisper model
            audio_path (str): Audio file path (can be .mp3, .wav, .m4a, etc.).
            Ej. 'audio_1.mp3'

        Raises:
            ValueError: Invalid language or model
        """
        if language not in self.whisper_languages:
            raise ValueError(
                f"Invalid language: {language}. Available languages are {self.whisper_languages}"
            )

        if model not in self.whisper_models:
            raise ValueError(
                f"Invalid model: {model}. Available models are {self.whisper_models}"
            )
        # Load the model
        model: whisper = whisper.load_model(model)

        # Transcribe the audio
        result = model.transcribe(audio_path, language=language)

        # text transcribe
        return result["text"]

    def faster_whisper_v1(self, audio_path: str, language: str, model: str = "tiny"):
        """
        Faster Whisper model

        Args:
            model (str): The whisper model
            audio_path (str): Audio file path (can be .mp3, .wav, .m4a, etc.).
            Ej. 'audio_1.mp3'

        Raises:
            ValueError: Invalid language or model
        """
        if language not in self.whisper_languages:
            raise ValueError(
                f"Invalid language: {language}. Available languages are {self.whisper_languages}"
            )

        if model not in self.whisper_models:
            raise ValueError(
                f"Invalid model: {model}. Available models are {self.whisper_models}"
            )

        model = WhisperModel(
            "base", compute_type="int8"
        )  # Puede ser int8, float16 o float32
        segments, info = model.transcribe(
            r"E:\data no borrar\documentos\Python proyecto\ProyectosJoel\audio\audios_examples\english_30s.wav",
            language="en",
        )

        for segment in segments:
            print(f"[{segment.start:.2f} - {segment.end:.2f}] {segment.text}")

    def faster_whisper(
        self,
        audio_path: str,
        language: str | None = None,
        model: str = "base",
        compute_type: Literal["int8", "float16", "float32"] = "int8",
        beam_size: int = 5,
        word_timestamps: bool = False,
        vad_filter: bool = True,
    ) -> str:
        """
        Audio transcription using faster-whisper (optimized)

        Parameters
        ----------
        1. audio_path (`str`):
                - Absolute path to the audio file. It is recommended to
                use audio files in WAV/PCM format for best performance.
        2. language (`str`, optional):
                - Language code of the audio to be processed.
                For example, 'es' for Spanish, 'en' for English.
                By default is 'en' (english).
        3. model (`str`, optional):
                - The whisper model to be used in the process.
                For example, 'tiny' for the lightest model
                By default is 'base'.
        4. compute_type (`str`, optional):
                - Numerical accuracy to be used by the model for calculations.
                float32: Highest precision (32 bits) but slower and more expensive.
                float16 (recommended): Medium precision (16 bits).
                int8: Low precision (8 integer bits) but faster and cheaper (for cpu).
                By default is 'int8' to run in CPU if CUDA is not available.
        5. beam_size (`int`, optional):
                - Beam search algorithm that controls how many possible word.
                sequences it considers simultaneously. (affects transcription accuracy).
                the higher this value, the higher the probability of identifying the correct words.
                By default is 5.
        6. compute_type (`str`, optional):
                - Numerical accuracy to be used by the model for calculations.
                float32: Highest precision (32 bits) but slower and more expensive.
                float16 (recommended): Medium precision (16 bits).
                int8: Low precision (8 integer bits) but faster and cheaper.
                By default is 'float16'.
        7. word_timestamps (`bool`, optional):
                - Controls whether timestamps are generated for each individual word.
                When True returns the start and end time of each word.
                By default is 'False'.
        8. vad_filter (`bool`, optional):
                - Automatically detects speech segments vs. silences. When true, filters
                out background noise and silences, reduces processing time.
                By default is 'True'.

        Returns:
            str: text transcribed
        """
        # validations
        if not os.path.exists(audio_path):
            raise FileNotFoundError(f"Archivo de audio no encontrado: {audio_path}")

        if model not in self.whisper_models:
            raise ValueError(
                f"Modelo inválido. Opciones: {', '.join(self.whisper_models)}"
            )

        if compute_type not in self.compute_types:
            raise ValueError(
                f"Tipo de computación inválido. Opciones: {', '.join(self.compute_types)}"
            )

        # Cargar modelo (con cache para reutilizar)
        model_key = f"{model}_{compute_type}"
        if model_key not in self.model_cache:
            self.model_cache[model_key] = WhisperModel(
                model_size_or_path=model,
                device=(  # int8 is a type of computation that can run on CPUs without penalty.
                    "cuda"
                    if compute_type != "int8" and torch.cuda.is_available()
                    else "cpu"
                ),  # Auto device selection
                compute_type=compute_type,
            )

        model_instance = self.model_cache[model_key]

        # Configurar parámetros de transcripción
        options = {
            "beam_size": beam_size,
            "vad_filter": vad_filter,
            "word_timestamps": word_timestamps,
            "language": language if language else None,  # None = detección automática
        }

        # Realizar transcripción
        segments, info = model_instance.transcribe(audio=audio_path, **options)

        # Construir resultado
        full_text = []
        for segment in segments:
            if word_timestamps:
                for word in segment.words:
                    full_text.append(f"[{word.start:.2f}-{word.end:.2f}] {word.word}")
            else:
                full_text.append(segment.text)

        return "\n".join(full_text)


def unit_test():
    txt = STTModels().faster_whisper(
        audio_path="audio.wav",
        # es: español , en: ingles, pt: portugues
        language="es",
        model="large", # large for better transcriptions
        compute_type="float32",
        beam_size=5,
    )

    print(txt)


if __name__ == "__main__":
    # Run unit test when executing this script
    unit_test()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

preprocessor_config.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

vocabulary.json: 0.00B [00:00, ?B/s]

model.bin:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

 Hola a todos y bienvenido de nuevo a mi canal.
 En este vídeo vamos a ver cómo puedes crear un indicador en la plataforma TradingView.
 Así que, ¡vamos allá!
 Antes de ir al grano, te dejo aquí abajo mis redes sociales,
 arrobaHobbyCode, me puedes encontrar aquí en YouTube,
 donde subimos un vídeo de forma semanal.
 También, sobre todo, en Instagram, donde comparto contenidos
 y explico historias sobre trading, trading algorítmico,
 criptomonedas y el mercado de acciones de forma diaria.
 Así que, abre el móvil ahora mismo y búscame en Instagram, arrobaHobbyCode.
 Y finalmente, también estamos en Twitch, en el que hacemos directos cada domingo
 a las 10 horas española, tanto en Twitch como en YouTube.
 Bien, hoy vamos directo ya a la plataforma de TradingView, tradingview.com.
 Aquí esta plataforma, para aquellos que no lo sepáis, es una plataforma
 donde muchos traders analizan los gráficos, ¿vale?
 Se conecta a muchísimas fuentes de datos.
 Actualmente tiene todos los activos que ex